In [37]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# class charges:
#     stt = 0.0/100
#     brokerage = 0.1/100
#     tc = 0.00/100
#     gst = 0/100
#     sd = 0./100
#     impact = 0.5/100

# class charges:
#     stt = 0.1/100
#     brokerage = 0.1/100
#     tc = 0.00322/100
#     gst = 18/100
#     sd = 0.015/100
#     impact = 0.3/100

class charges:
    stt = 0./100
    brokerage = 0.0/100
    tc = 0.0/100
    gst = 0/100
    sd = 0.0/100
    impact = 0./100

def generalRebalanceDay1(portfolio, currHoldings, rebalanceCounter):

    entry_stocks = set(portfolio.set_index("Symbol").index.tolist()) - set(currHoldings.index.tolist())
    hold_stocks = set(portfolio.set_index("Symbol").index.tolist()).intersection(set(currHoldings.index.tolist()))
    exit_stocks = set(currHoldings.index.tolist()) - set(portfolio.set_index("Symbol").index.tolist())
    exit_stocks.remove("Cash")
    hold_stocks.add("Cash")

    currHoldings = pd.concat([currHoldings, portfolio.set_index("Symbol")[["Weight"]]], axis = 1)

    currHoldings.loc[list(entry_stocks), "HoldingStatus"] = "Enter"
    currHoldings.loc[list(hold_stocks), "HoldingStatus"] = "Hold"
    currHoldings.loc[list(exit_stocks), "HoldingStatus"] = "Exit"

    currHoldings[["PortfolioWeight", "Weight"]] = currHoldings[["PortfolioWeight", "Weight"]].fillna(0)

    price = pd.concat([currHoldings[["Price"]].dropna(), portfolio.set_index("Symbol")[["Price"]]], axis = 0)

    currHoldings.drop(columns = ["Price"], inplace = True)

    currHoldings = pd.concat([currHoldings, price[~price.index.duplicated()]], axis = 1)

    currHoldings.loc["Cash", "Price"] = currHoldings.loc["Cash", "Value"]

    currHoldings["Value"] = (currHoldings["Qty"]*currHoldings["Price"]).fillna(0)

    currHoldings["PortfolioWeight"] = currHoldings["Value"].div(currHoldings["Value"].sum()).fillna(0)
    currHoldings["TgtWeight"] = currHoldings["Weight"] - currHoldings["PortfolioWeight"]

    currHoldings["Sign"] = np.sign(currHoldings["TgtWeight"])

    exit_stocks = (currHoldings["HoldingStatus"] == "Exit")
    trim_stocks = (currHoldings["HoldingStatus"] == "Hold") & (currHoldings["Sign"] == -1)
    # add_stocks =  (currentHoldings["HoldingStatus"] == "Hold") & (currentHoldings["Sign"] == 1)
    # new_stocks =  (currentHoldings["HoldingStatus"] == "Enter")

    exit_value = currHoldings.loc[exit_stocks]["Value"].sum()
    trim_value = np.abs((currHoldings.loc[trim_stocks].drop("Cash")["TgtWeight"]*(currHoldings["Value"].sum())).sum())
    rebalance_value = np.round(exit_value + trim_value, 2)
    
    currHoldings.loc[exit_stocks, "SellValue"] = (rebalance_value/3)/exit_stocks.sum()

    # Calculating the Real Life Charges Scenario.
    currHoldings["ImpPrice"] = currHoldings["Price"]*(1-charges.impact)
    currHoldings["brokerage"] = (currHoldings["ImpPrice"])*(charges.brokerage)
    currHoldings["gst"] = currHoldings["brokerage"]*(charges.gst)
    currHoldings["stt"] = currHoldings["ImpPrice"]*(charges.stt)
    currHoldings["stampduty"] = currHoldings["ImpPrice"]*(charges.sd)
    currHoldings["txncharge"] = currHoldings["ImpPrice"]*(charges.tc)
    currHoldings["Charges"] = currHoldings[["brokerage", "gst", "stt", "stampduty","txncharge"]].sum(axis = 1)
    currHoldings["SellPrice"] = (currHoldings["ImpPrice"] - currHoldings["Charges"]).round(2)

    currHoldings["SellQty"] = currHoldings["SellValue"] // currHoldings["SellPrice"]

    # # Calculate the Total Charges
    currHoldings[["brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]] = currHoldings[["brokerage", "gst", "stt","stampduty","txncharge", "Charges"]].mul(currHoldings["SellQty"], axis = 0)

    ## Charges Paid
    chargesPaid = currHoldings[["Qty", "ImpPrice", "brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]].copy()

    currHoldings["SellValue"] = currHoldings["SellQty"] * currHoldings["SellPrice"]

    currHoldings.loc[exit_stocks, "Qty"] = currHoldings.loc[exit_stocks, "Qty"] - currHoldings.loc[exit_stocks, "SellQty"]

    cashGenerated = np.abs(currHoldings["SellValue"].sum())

    currHoldings["Value"] = currHoldings["Qty"]*currHoldings["Price"]
    currHoldings.loc["Cash", "Value"] += cashGenerated

    currHoldings = currHoldings[["Qty", "Value","HoldingStatus"]].copy()

    currHoldings["PortfolioWeight"] = currHoldings["Value"].div(currHoldings["Value"].sum())

    currHoldings[["Qty", "Value", "PortfolioWeight"]] = currHoldings[["Qty", "Value", "PortfolioWeight"]].fillna(0)
    
    return currHoldings, chargesPaid

def generalRebalanceDay7Buying(currHoldings, rebalanceCounter):

    currHoldings["Value"] = (currHoldings["Qty"]*currHoldings["Open"]).fillna(0)

    currHoldings["PortfolioWeight"] = currHoldings["Value"].div(currHoldings["Value"].sum()).fillna(0)
    currHoldings["TgtWeight"] = currHoldings["Weight"] - currHoldings["PortfolioWeight"]

    currHoldings["Sign"] = np.sign(currHoldings["TgtWeight"])

    new_stocks =  (currHoldings["HoldingStatus"] == "Enter") 
    add_stocks = (currHoldings["HoldingStatus"] == "Hold") & (currHoldings["Sign"] == 1) & (currHoldings.index != "Cash")

    cashToDeploy = currHoldings.loc["Cash", "Value"]

    currHoldings.loc[(new_stocks|add_stocks), "TgtWeight"] = currHoldings.loc[(new_stocks|add_stocks), "TgtWeight"] / currHoldings.loc[(new_stocks|add_stocks), "TgtWeight"].sum()

    currHoldings.loc[(new_stocks|add_stocks), "BuyValue"] = cashToDeploy* currHoldings.loc[(new_stocks|add_stocks), "TgtWeight"]

    # Calculating the Real Life Charges Scenario.
    currHoldings["ImpPrice"] = currHoldings["Open"]*(1+charges.impact)
    currHoldings["brokerage"] = (currHoldings["ImpPrice"])*(charges.brokerage)
    currHoldings["gst"] = currHoldings["brokerage"]*(charges.gst)
    currHoldings["stt"] = currHoldings["ImpPrice"]*(charges.stt)
    currHoldings["stampduty"] = currHoldings["ImpPrice"]*(charges.sd)
    currHoldings["txncharge"] = currHoldings["ImpPrice"]*(charges.tc)
    currHoldings["Charges"] = currHoldings[["brokerage", "gst", "stt", "stampduty","txncharge"]].sum(axis = 1)
    currHoldings["BuyPrice"] = (currHoldings["ImpPrice"] + currHoldings["Charges"]).round(2)

    # Calculate the number of shares to buy for each symbol based on equal distribution of cash
    currHoldings["BuyQty"] = currHoldings["BuyValue"] // currHoldings["BuyPrice"]

    # Calculate the total value for each symbol (Qty * Price)
    currHoldings["BuyValue"] = currHoldings["BuyQty"] * currHoldings["BuyPrice"]
    
    # Calculate the Total Charges
    currHoldings[["brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]] = currHoldings[["brokerage", "gst", "stt","stampduty","txncharge", "Charges"]].mul(currHoldings["BuyQty"], axis = 0)

    ## Charges Paid
    chargesPaid = currHoldings[["BuyQty", "ImpPrice", "brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]].copy()

    currHoldings["Qty"] = currHoldings[["Qty", "BuyQty"]].sum(axis = 1)

    # Calculate the remaining cash and add it as a new row to the current holdings
    cashRemaining = cashToDeploy - currHoldings["BuyValue"].sum()
    currHoldings.loc["Cash", ["Qty", "Value", "Open", "Price"]] = [1, cashRemaining,cashRemaining,cashRemaining]
    
    currHoldings["Value"] = currHoldings["Qty"]*currHoldings["Price"]
    currHoldings["PortfolioWeight"] = currHoldings["Value"].div(currHoldings["Value"].sum())
    
    currHoldings.drop(columns = ["BuyQty", "ImpPrice", "brokerage", "gst", "stt", "stampduty", "txncharge", "Charges", "BuyValue", "BuyPrice"], inplace = True)

    currHoldings = currHoldings[["Qty", "Value", "PortfolioWeight"]].copy()

    return currHoldings, chargesPaid

def generalRebalanceDay3Buying(currHoldings, rebalanceCounter):

    currHoldings["Value"] = (currHoldings["Qty"]*currHoldings["Open"]).fillna(0)

    currHoldings["PortfolioWeight"] = currHoldings["Value"].div(currHoldings["Value"].sum()).fillna(0)
    currHoldings["TgtWeight"] = currHoldings["Weight"] - currHoldings["PortfolioWeight"]

    currHoldings["Sign"] = np.sign(currHoldings["TgtWeight"])

    new_stocks =  (currHoldings["HoldingStatus"] == "Enter") 

    cashToDeploy = currHoldings.loc["Cash", "Value"]

    # Calculating the Real Life Charges Scenario.
    currHoldings["ImpPrice"] = currHoldings["Open"]*(1+charges.impact)
    currHoldings["brokerage"] = (currHoldings["ImpPrice"])*(charges.brokerage)
    currHoldings["gst"] = currHoldings["brokerage"]*(charges.gst)
    currHoldings["stt"] = currHoldings["ImpPrice"]*(charges.stt)
    currHoldings["stampduty"] = currHoldings["ImpPrice"]*(charges.sd)
    currHoldings["txncharge"] = currHoldings["ImpPrice"]*(charges.tc)
    currHoldings["Charges"] = currHoldings[["brokerage", "gst", "stt", "stampduty","txncharge"]].sum(axis = 1)
    currHoldings["BuyPrice"] = (currHoldings["ImpPrice"] + currHoldings["Charges"]).round(2)

    # Calculate the number of shares to buy for each symbol based on equal distribution of cash
    currHoldings.loc[new_stocks, "BuyQty"] = (cashToDeploy / new_stocks.sum()) // currHoldings.loc[new_stocks, "BuyPrice"]
    
    # Calculate the total value for each symbol (Qty * Price)
    currHoldings["BuyValue"] = currHoldings["BuyQty"] * currHoldings["BuyPrice"]
    
    # Calculate the Total Charges
    currHoldings[["brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]] = currHoldings[["brokerage", "gst", "stt","stampduty","txncharge", "Charges"]].mul(currHoldings["BuyQty"], axis = 0)

    ## Charges Paid
    chargesPaid = currHoldings[["BuyQty", "ImpPrice", "brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]].copy()

    currHoldings["Qty"] = currHoldings[["Qty", "BuyQty"]].sum(axis = 1)

    # Calculate the remaining cash and add it as a new row to the current holdings
    cashRemaining = cashToDeploy - currHoldings["BuyValue"].sum()
    currHoldings.loc["Cash", ["Qty", "Value", "Open", "Price"]] = [1, cashRemaining,cashRemaining,cashRemaining]
    currHoldings["Value"] = currHoldings["Qty"]*currHoldings["Open"]
    currHoldings["PortfolioWeight"] = currHoldings["Value"].div(currHoldings["Value"].sum())
    
    currHoldings.drop(columns = ["BuyQty", "ImpPrice", "brokerage", "gst", "stt", "stampduty", "txncharge", "Charges", "BuyValue", "BuyPrice"], inplace = True)

    return currHoldings, chargesPaid

def generalRebalanceDay3Selling(currHoldings, rebalanceCounter):

    currHoldings["Value"] = (currHoldings["Qty"]*currHoldings["Price"]).fillna(0)

    currHoldings["PortfolioWeight"] = currHoldings["Value"].div(currHoldings["Value"].sum()).fillna(0)
    currHoldings["TgtWeight"] = currHoldings["Weight"] - currHoldings["PortfolioWeight"]

    currHoldings["Sign"] = np.sign(currHoldings["TgtWeight"])

    exit_stocks = (currHoldings["HoldingStatus"] == "Exit")
    trim_stocks = (currHoldings["HoldingStatus"] == "Hold") & (currHoldings["Sign"] == -1)

    exit_value = currHoldings.loc[exit_stocks]["Value"].sum()
    trim_value = np.abs((currHoldings.loc[trim_stocks].drop("Cash")["TgtWeight"]*(currHoldings["Value"].sum())).sum())
    rebalance_value = np.round(exit_value + trim_value, 2)

    currHoldings.loc[exit_stocks, "SellValue"] = (rebalance_value/2)/exit_stocks.sum()

    # Calculating the Real Life Charges Scenario.
    currHoldings["ImpPrice"] = currHoldings["Price"]*(1-charges.impact)
    currHoldings["brokerage"] = (currHoldings["ImpPrice"])*(charges.brokerage)
    currHoldings["gst"] = currHoldings["brokerage"]*(charges.gst)
    currHoldings["stt"] = currHoldings["ImpPrice"]*(charges.stt)
    currHoldings["stampduty"] = currHoldings["ImpPrice"]*(charges.sd)
    currHoldings["txncharge"] = currHoldings["ImpPrice"]*(charges.tc)
    currHoldings["Charges"] = currHoldings[["brokerage", "gst", "stt", "stampduty","txncharge"]].sum(axis = 1)
    currHoldings["SellPrice"] = (currHoldings["ImpPrice"] - currHoldings["Charges"]).round(2)

    currHoldings["SellQty"] = currHoldings["SellValue"] // currHoldings["SellPrice"]

    # # Calculate the Total Charges
    currHoldings[["brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]] = currHoldings[["brokerage", "gst", "stt","stampduty","txncharge", "Charges"]].mul(currHoldings["SellQty"], axis = 0)

    ## Charges Paid
    chargesPaid = currHoldings[["SellQty", "ImpPrice", "brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]].copy()

    currHoldings["SellValue"] = currHoldings["SellQty"] * currHoldings["SellPrice"]
    
    currHoldings.loc[exit_stocks, "Qty"] = currHoldings.loc[exit_stocks, "Qty"] - currHoldings.loc[exit_stocks, "SellQty"]

    cashGenerated = np.abs(currHoldings["SellValue"].sum())

    currHoldings["Value"] = currHoldings["Qty"]*currHoldings["Price"]
    currHoldings.loc["Cash", "Value"] += cashGenerated

    currHoldings = currHoldings[["Qty", "Value","HoldingStatus"]].copy()

    currHoldings["PortfolioWeight"] = currHoldings["Value"].div(currHoldings["Value"].sum())

    currHoldings[["Qty", "Value", "PortfolioWeight"]] = currHoldings[["Qty", "Value", "PortfolioWeight"]].fillna(0)
    
    return currHoldings, chargesPaid    

def generalRebalanceDay5Selling(currHoldings, rebalanceCounter):

    currHoldings["Value"] = (currHoldings["Qty"]*currHoldings["Price"]).fillna(0)

    currHoldings["PortfolioWeight"] = currHoldings["Value"].div(currHoldings["Value"].sum()).fillna(0)
    currHoldings["TgtWeight"] = currHoldings["Weight"] - currHoldings["PortfolioWeight"]

    currHoldings["Sign"] = np.sign(currHoldings["TgtWeight"])

    exit_stocks = (currHoldings["HoldingStatus"] == "Exit")
    trim_stocks = (currHoldings["HoldingStatus"] == "Hold") & (currHoldings["Sign"] == -1) & (currentHoldings.index != "Cash")

    # Calculating the Real Life Charges Scenario.
    currHoldings["ImpPrice"] = currHoldings["Price"]*(1-charges.impact)
    currHoldings["brokerage"] = (currHoldings["ImpPrice"])*(charges.brokerage)
    currHoldings["gst"] = currHoldings["brokerage"]*(charges.gst)
    currHoldings["stt"] = currHoldings["ImpPrice"]*(charges.stt)
    currHoldings["stampduty"] = currHoldings["ImpPrice"]*(charges.sd)
    currHoldings["txncharge"] = currHoldings["ImpPrice"]*(charges.tc)
    currHoldings["Charges"] = currHoldings[["brokerage", "gst", "stt", "stampduty","txncharge"]].sum(axis = 1)
    currHoldings["SellPrice"] = (currHoldings["ImpPrice"] - currHoldings["Charges"]).round(2)

    currHoldings.loc[exit_stocks, "SellQty"] = currHoldings.loc[exit_stocks, "Qty"]
    currHoldings.loc[exit_stocks, "SellValue"] = currHoldings.loc[exit_stocks, "SellPrice"] * currHoldings.loc[exit_stocks, "Qty"]

    currHoldings.loc[trim_stocks, "SellValue"] = np.round(np.abs(currHoldings[trim_stocks]["TgtWeight"]*currHoldings["Value"].sum()), 2)
    currHoldings.loc[trim_stocks,"SellQty"] = currHoldings.loc[trim_stocks, "SellValue"] // currHoldings.loc[trim_stocks, "SellPrice"] 

    # # Calculate the Total Charges
    currHoldings[["brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]] = currHoldings[["brokerage", "gst", "stt","stampduty","txncharge", "Charges"]].mul(currHoldings["SellQty"], axis = 0)

    ## Charges Paid
    chargesPaid = currHoldings[["SellQty", "ImpPrice", "brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]].copy()

    currHoldings["SellValue"] = currHoldings["SellQty"] * currHoldings["SellPrice"]
    
    currHoldings.loc[exit_stocks, "Qty"] = currHoldings.loc[exit_stocks, "Qty"] - currHoldings.loc[exit_stocks, "SellQty"]
    currHoldings.loc[trim_stocks, "Qty"] = currHoldings.loc[trim_stocks, "Qty"] - currHoldings.loc[trim_stocks, "SellQty"]

    cashGenerated = np.abs(currHoldings["SellValue"].sum())
    
    currHoldings["Value"] = currHoldings["Qty"]*currHoldings["Price"]
    currHoldings.loc["Cash", "Value"] += cashGenerated  

    currHoldings = currHoldings[["Qty", "Value","HoldingStatus"]].copy()

    currHoldings["PortfolioWeight"] = currHoldings["Value"].div(currHoldings["Value"].sum())

    currHoldings[["Qty", "Value", "PortfolioWeight"]] = currHoldings[["Qty", "Value", "PortfolioWeight"]].fillna(0)

    currHoldings = currHoldings[currHoldings["HoldingStatus"] != "Exit"].copy()
    
    return currHoldings, chargesPaid
    
def get_financial_year(date):
    """
    Determines the financial year for a given date.
    
    In many regions, the financial year starts on April 1st and ends on March 31st of the following year.
    This function calculates the financial year based on this assumption.

    Parameters:
    date (datetime.date or similar) : The date for which the financial year needs to be determined.

    Returns:
    str : The financial year in the format "FYYYYY-YYYY", where the first YYYY is the start year and 
          the second YYYY is the end year of the financial year.
    """
    # Extract the year from the given date
    year = date.year
    
    # Check if the date is before April (month < 4)
    if date.month < 4:
        # If before April, it belongs to the previous financial year
        return f"FY{year-1}-{year}"
    else:
        # If April or later, it belongs to the current financial year
        return f"FY{year}-{year+1}"

def performInitialRebalanceNoComments(portfolio, currentHoldings, capital, N, rebalanceCounter):

    if rebalanceCounter == 0:
        cashToDeploy = capital / N
        portfolio.set_index("Symbol", inplace = True)
        portfolio["Qty"] = (cashToDeploy/len(portfolio)) // portfolio["Price"]
        portfolio["Value"] = portfolio["Qty"]*portfolio["Price"]
        currentHoldings = portfolio[[ "Qty", "Value"]].copy()
        currentHoldings.loc["Cash", ["Qty", "Value"]] = [1, capital - currentHoldings["Value"].sum()]
        currentHoldings["PortfolioWeight"] = currentHoldings["Value"].div(currentHoldings["Value"].sum())
    else:
        cashToDeploy = currentHoldings.loc["Cash", "Value"] / (N - rebalanceCounter)
        currentHoldings = pd.concat([currentHoldings[["Qty", "Value"]], portfolio.set_index("Symbol")], axis = 1)
        currentHoldings.loc["Cash", "Price"] = currentHoldings.loc["Cash", "Value"]

        currentHoldings["CurrWt"] = (currentHoldings["Qty"]*currentHoldings["Price"]).div((currentHoldings["Qty"]*currentHoldings["Price"]).sum())
        currentHoldings["TgtWt"] = (currentHoldings["Weight"] - currentHoldings["CurrWt"]) / (N - rebalanceCounter)
        currentHoldings["TgtWt"] = currentHoldings["TgtWt"] / currentHoldings["TgtWt"].sum()

        currentHoldings["BuyQty"] = (cashToDeploy/len(portfolio)) // currentHoldings["Price"]
        currentHoldings["BuyValue"] = currentHoldings["BuyQty"]*currentHoldings["Price"]

        currentHoldings["Qty"] = currentHoldings[["Qty", "BuyQty"]].sum(axis = 1)
        currentHoldings["Value"] = currentHoldings["Qty"] * currentHoldings["Price"]

        cashRemaining = currentHoldings.loc["Cash", "Value"] - cashToDeploy + (cashToDeploy - currentHoldings["BuyValue"].sum())
        currentHoldings = currentHoldings[["Qty", "Value"]]

        currentHoldings.loc["Cash", ["Qty", "Value"]] = [1, cashRemaining]
        currentHoldings["PortfolioWeight"] = currentHoldings["Value"].div(currentHoldings["Value"].sum())

    return  currentHoldings

def performInitialRebalance(portfolio, currentHoldings, capital, N, rebalanceCounter):
    # Check if it's the first initial rebalance day(rebalanceCounter == 0)
    if rebalanceCounter == 0:
        # Calculate the cash to be deployed for this rebalance
        cashToDeploy = capital / N
        
        # Set the index of the portfolio dataframe to "Symbol" for easier lookups
        portfolio.set_index("Symbol", inplace=True)

        # Calculating the Real Life Charges Scenario.
        portfolio["ImpPrice"] = portfolio["Price"]*(1+charges.impact)
        portfolio["brokerage"] = (portfolio["ImpPrice"])*(charges.brokerage)
        portfolio["gst"] = portfolio["brokerage"]*(charges.gst)
        portfolio["stt"] = portfolio["ImpPrice"]*(charges.stt)
        portfolio["stampduty"] = portfolio["ImpPrice"]*(charges.sd)
        portfolio["txncharge"] = portfolio["ImpPrice"]*(charges.tc)
        portfolio["Charges"] = portfolio[["brokerage", "gst", "stt", "stampduty","txncharge"]].sum(axis = 1)
        portfolio["BuyPrice"] = portfolio[["ImpPrice", "Charges"]].sum(axis = 1).round(2)
        
        # Calculate the number of shares to buy for each symbol based on equal distribution of cash
        portfolio["Qty"] = (cashToDeploy / len(portfolio)) // portfolio["BuyPrice"]
        # portfolio["Qty"] = (cashToDeploy / len(portfolio)) // (portfolio["BuyPrice"] + 1e-10)
        # Calculate the total value for each symbol (Qty * Price)
        portfolio["Value"] = portfolio["Qty"] * portfolio["BuyPrice"]
        
        # Calculate the Total Charges
        portfolio[["brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]] = portfolio[["brokerage", "gst", "stt","stampduty","txncharge", "Charges"]].mul(portfolio["Qty"], axis = 0)

        ## Charges Paid
        chargesPaid = portfolio[["Qty", "ImpPrice", "brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]].copy()

        # Initialize the current holdings with the portfolio's quantity and value
        currentHoldings = portfolio[["Qty", "Value"]].copy()
        
        # Calculate the remaining cash and add it as a new row to the current holdings
        currentHoldings.loc["Cash", ["Qty", "Value"]] = [1, capital - currentHoldings["Value"].sum()]
        
        # Calculate the portfolio weights based on the value of each holding
        currentHoldings["PortfolioWeight"] = currentHoldings["Value"].div(currentHoldings["Value"].sum())
    
    # If it's not the first initial rebalance day
    else:
        # Calculate the cash to deploy for the remaining rebalancing periods
        cashToDeploy = currentHoldings.loc["Cash", "Value"] / (N - rebalanceCounter)
        
        # Merge the current holdings with the new portfolio stock price data based on the symbol
        currentHoldings = pd.concat([currentHoldings[["Qty", "Value"]], portfolio.set_index("Symbol")], axis=1)
        
        # Set the current value for the cash row
        currentHoldings.loc["Cash", "Price"] = currentHoldings.loc["Cash", "Value"]

        # Calculate the current portfolio weight (based on current holdings)
        currentHoldings["CurrWt"] = (currentHoldings["Qty"] * currentHoldings["Price"]).div((currentHoldings["Qty"] * currentHoldings["Price"]).sum())
        
        # Calculate the target portfolio weight for each symbol (adjusted for remaining rebalances)
        currentHoldings["TgtWt"] = (currentHoldings["Weight"] - currentHoldings["CurrWt"]) / (N - rebalanceCounter)
        
        # Normalize the target weights to ensure they sum to 1
        currentHoldings["TgtWt"] = currentHoldings["TgtWt"] / currentHoldings["TgtWt"].sum()

        currentHoldings["BuyValue"]= (currentHoldings["TgtWt"]*cashToDeploy).round(2)

        # Calculating the Real Life Charges Scenario.
        currentHoldings["ImpPrice"] = currentHoldings["Price"]*(1+charges.impact)
        currentHoldings["brokerage"] = (currentHoldings["ImpPrice"])*(charges.brokerage)
        currentHoldings["gst"] = currentHoldings["brokerage"]*(charges.gst)
        currentHoldings["stt"] = currentHoldings["ImpPrice"]*(charges.stt)
        currentHoldings["stampduty"] = currentHoldings["ImpPrice"]*(charges.sd)
        currentHoldings["txncharge"] = currentHoldings["ImpPrice"]*(charges.tc)
        currentHoldings["Charges"] = currentHoldings[["brokerage", "gst", "stt", "stampduty","txncharge"]].sum(axis = 1)
        currentHoldings["BuyPrice"] = currentHoldings[["ImpPrice", "Charges"]].sum(axis = 1).round(2)

        # Calculate the quantity of shares to buy for each symbol based on the cash to deploy
        currentHoldings["BuyQty"] = currentHoldings["BuyValue"] // currentHoldings["BuyPrice"]
        
        # Calculate the value of the shares to be bought
        currentHoldings["BuyValue"] = currentHoldings["BuyQty"] * currentHoldings["BuyPrice"]

        # Update the total quantity and value for each symbol after the rebalance
        currentHoldings["Qty"] = currentHoldings[["Qty", "BuyQty"]].sum(axis=1)
        currentHoldings["Value"] = currentHoldings["Qty"] * currentHoldings["Price"]

        # Calculate the Total Charges
        currentHoldings[["brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]] = currentHoldings[["brokerage", "gst", "stt","stampduty","txncharge", "Charges"]].mul(currentHoldings["BuyQty"], axis = 0)
        ## Charges Paid
        chargesPaid = currentHoldings[["BuyQty", "ImpPrice", "brokerage", "gst", "stt", "stampduty", "txncharge", "Charges"]].copy()

        # Calculate the remaining cash after the rebalance
        cashRemaining = currentHoldings.loc["Cash", "Value"] - cashToDeploy + (cashToDeploy - currentHoldings["BuyValue"].sum())
        
        # Keep only the updated quantity and value columns in the current holdings
        currentHoldings = currentHoldings[["Qty", "Value"]]

        # Update the cash row with the remaining cash
        currentHoldings.loc["Cash", ["Qty", "Value"]] = [1, cashRemaining]
        
        # Recalculate the portfolio weights after the rebalance
        currentHoldings["PortfolioWeight"] = currentHoldings["Value"].div(currentHoldings["Value"].sum())

    # Return the updated current holdings
    return currentHoldings, chargesPaid

def updateDailyNAV(currentHoldings, price_data, liveDate):

    # Extract the closing prices for the current live date from the price data
    price = price_data[price_data["Date"] == liveDate].set_index("Symbol")[["Close"]]

    # Merge the current holdings with the price data to get the latest close prices for each symbol
    currentHoldings = pd.merge(currentHoldings, price, left_index=True, right_index=True, how="left")

    # Set the "Close" price for cash to be the current cash value (as cash doesn't change in value)
    currentHoldings.loc["Cash", "Close"] = currentHoldings.loc["Cash", "Value"]

    # Recalculate the value of each holding (Quantity * Close Price)
    currentHoldings["Value"] = currentHoldings["Qty"] * currentHoldings["Close"]

    # Recalculate the portfolio weight based on the updated values
    currentHoldings["PortfolioWeight"] = currentHoldings["Value"].div(currentHoldings["Value"].sum())

    # Drop the "Close" column as it's no longer needed after the value update
    currentHoldings.drop(columns=["Close"], inplace=True)

    # Return the updated current holdings
    return currentHoldings

def performGeneralRebalance(portfolio, currHoldings, rebalanceCounter):

    if rebalanceCounter == 0:
        
        entry_stocks = set(portfolio.set_index("Symbol").index.tolist()) - set(currHoldings.index.tolist())
        hold_stocks = set(portfolio.set_index("Symbol").index.tolist()).intersection(set(currHoldings.index.tolist()))
        exit_stocks = set(currHoldings.index.tolist()) - set(portfolio.set_index("Symbol").index.tolist())
        exit_stocks.remove("Cash")
        hold_stocks.add("Cash")

        currHoldings = pd.concat([currHoldings, portfolio.set_index("Symbol")[["Weight"]]], axis = 1)

        currHoldings.loc[list(entry_stocks), "HoldingStatus"] = "Enter"
        currHoldings.loc[list(hold_stocks), "HoldingStatus"] = "Hold"
        currHoldings.loc[list(exit_stocks), "HoldingStatus"] = "Exit"

        currHoldings[["PortfolioWeight", "Weight"]] = currHoldings[["PortfolioWeight", "Weight"]].fillna(0)

        price = pd.concat([currHoldings[["Price"]].dropna(), portfolio.set_index("Symbol")[["Price"]]], axis = 0)

        currHoldings.drop(columns = ["Price"], inplace = True)

        currHoldings = pd.concat([currHoldings, price[~price.index.duplicated()]], axis = 1)

        currHoldings.loc["Cash", "Price"] = currHoldings.loc["Cash", "Value"]

        currHoldings["Value"] = (currHoldings["Qty"]*currHoldings["Price"]).fillna(0)

        currHoldings["PortfolioWeight"] = currHoldings["Value"].div(currHoldings["Value"].sum()).fillna(0)
        currHoldings["TgtWeight"] = currHoldings["Weight"] - currHoldings["PortfolioWeight"]

        currHoldings["Sign"] = np.sign(currHoldings["TgtWeight"])

        currHoldings["SellQty"] = np.where((currHoldings["HoldingStatus"].isin(["Hold", "Exit"])) & (currHoldings["Sign"] == -1)& (currHoldings.index != "Cash"),
                                            (currHoldings["TgtWeight"]/3)*currHoldings["Value"].sum() // currHoldings["Price"], np.nan)
        currHoldings["SellValue"] = currHoldings["SellQty"]*currHoldings["Price"]

        currHoldings["Qty1"] = currHoldings["Qty"]+currHoldings["SellQty"].fillna(0)

        cashGenerated = np.abs(currHoldings["SellValue"].sum())

        currHoldings["Value"] = currHoldings["Qty"]*currHoldings["Price"]
        currHoldings.loc["Cash", "Value"] += cashGenerated

        currHoldings = currHoldings[["Qty", "Value","HoldingStatus"]].copy()

        currHoldings["PortfolioWeight"] = currHoldings["Value"].div(currHoldings["Value"].sum())

        currHoldings[["Qty", "Value", "PortfolioWeight"]] = currHoldings[["Qty", "Value", "PortfolioWeight"]].fillna(0)
    else:
        pass


    return currHoldings

In [39]:
# allPortfolio = pd.read_csv("QualityInverseVol.csv", index_col = 0)
# allPortfolio

In [40]:
# Assuming mapping and price_data are already loaded as you mentioned
# mapping = pd.read_excel("company_master_mapping.xlsx")
price_data = pd.read_csv('stockPriceData.csv')
# etf_indices = pd.read_csv('etf_indices_18_2_2025.csv')
# price_data = price_data[~(price_data['Symbol'].isin(etf_indices['Symbol']))]
price_data['Date'] = pd.to_datetime(price_data['Date'])

# # Create a dictionary mapping from SYMBOL_NSE to SYMBOL_CM
# mapping = dict(zip(mapping["SYMBOL_NSE"], mapping["SYMBOL_CM"]))
# price_data["Symbol"] = price_data["Symbol"].replace(mapping)

# Filter price data for dates after '2006-01-01' and drop duplicates
price_data = price_data[price_data['Date'] >= '2006']
price_data.drop_duplicates(['Date', 'Symbol'], inplace=True)

# Convert 'Date' to datetime type
price_data['Date'] = pd.to_datetime(price_data['Date'], format='%d-%m-%Y')

# Optionally, format the Date column to 'YYYY-MM-DD' if you need
price_data['Date'] = price_data['Date'].dt.strftime('%Y-%m-%d')
price_data['Date'] = pd.to_datetime(price_data['Date'])

In [41]:
# Convert the "Date" column to datetime format
price_data["Date"] = pd.to_datetime(price_data["Date"])
price_data["Open"] = price_data[["Open", "High", "Low", "Close"]].mean(axis = 1)

# allPortfolio = pd.read_excel("stock_lists.xlsx", sheet_name = 'MCAP_300_TOP_10_FREQ_2')
allPortfolio = pd.read_csv("Trinity.csv")

# #Date error solution
# allPortfolio['Date'] = pd.to_datetime(allPortfolio['Date'], format='%d-%m-%Y')

# # Optionally, format the Date column to 'YYYY-MM-DD' if you need
# allPortfolio['Date'] = allPortfolio['Date'].dt.strftime('%Y-%m-%d')
allPortfolio['Date'] = pd.to_datetime(allPortfolio['Date'])

#To provide custom weights, uncomment the next line
# allPortfolio.rename(columns = {'VolWeight' : "Weight"}, inplace = True)
# allPortfolio.drop(columns = "Weight", inplace = True)

# allPortfolio = allPortfolio[allPortfolio["Date"].dt.month%2 != 0].reset_index(drop = True)

# Keep only the "Date" and "Symbol" columns from the portfolio
# allPortfolio = allPortfolio[["Date", "Symbol"]].copy()

# Define the initial capital as 100 million
capital = 10_00_00_000

# Set the initial rebalance period to 3 days
initialRebalancePeriod = 3

# Initialize flags to track the status of the initial rebalance
initialRebalanceRunning = False  # Track whether the initial rebalance is currently running
initialRebalanceDone = False  # Track whether the initial rebalance is completed

# Initialize flags to track the status of the general rebalance

generalRebalanceRunning = False  # Track whether the general rebalance is currently running
generalRebalanceDone = False  # Track whether the general rebalance is completed

# Set the regular rebalance period to 6 days
regularRebalancePeriod = 6

# Initialize a flag to track if weights have been assigned for the portfolio
weightsGiven = True

# Initialize a variable to hold the current portfolio holdings (to be populated later)

currentHoldings = None

# Initialize a counter to track the number of rebalances performed
rebalanceCounter = 0

In [42]:
# Filter price_data to include only the dates that are on or after the earliest date in allPortfolio
price_data = price_data[price_data["Date"] >= allPortfolio["Date"].min()]

# Filter price_data to include only the symbols present in allPortfolio
price_data = price_data[price_data["Symbol"].isin(allPortfolio["Symbol"].unique().tolist())]

# Sort the filtered price_data by Date and reset the index
price_data = price_data = price_data.sort_values(["Date", "Mcap"], ascending = [True, False]).reset_index(drop = True)

# Sort allPortfolio by Date and reset the index
allPortfolio = allPortfolio.sort_values("Date").reset_index(drop=True)

# Check if portfolio weights have not yet been assigned
if not weightsGiven:
    # Assign equal weights to all symbols for each date in the portfolio
    # Each symbol gets a weight of 1 divided by the total number of symbols on that date
    allPortfolio["Weight"] = 1 / allPortfolio.groupby("Date").transform(lambda x: x.count())

# Add a "FY" (Financial Year) column to the price data by applying the function get_financial_year to the "Date" column
price_data["FY"] = price_data["Date"].apply(get_financial_year)

# Add a "FY" (Financial Year) column to the portfolio data by applying the function get_financial_year to the "Date" column
allPortfolio["FY"] = allPortfolio["Date"].apply(get_financial_year)

In [43]:
# Get a list of unique financial years from the "FY" column in the price data
financialYears = price_data["FY"].unique().tolist()

# Get a list of unique rebalance dates from the "Date" column in the portfolio data
# rebalanceDates = allPortfolio["Date"].unique().tolist()

rebalanceDates = pd.to_datetime(allPortfolio["Date"].unique()).tolist()

In [44]:
allPortfolio['Weight'].max()

np.float64(0.04)

In [45]:
rebalanceCounter = 0
initialRebalanceRunning = False
initialRebalanceDone = False
generalRebalanceRunning = False
generalRebalanceDone = False
initialRebalanceDone = False
initialRebalanceRunning = False

nav = list()
transactionalCharges = list()


writer = pd.ExcelWriter("Rebalances.xlsx")


for fyYear in tqdm(financialYears):
    fyDates = price_data[price_data["FY"] == fyYear]["Date"].unique().tolist()

    for liveDate in fyDates:

        if (liveDate in rebalanceDates) and (not initialRebalanceDone) and (rebalanceCounter == 0):
            
            rebalancePortfolio = allPortfolio[allPortfolio["Date"] == liveDate].reset_index(drop = True)
            rebalancePortfolio = pd.merge(rebalancePortfolio, price_data[["Date", "Symbol", "Close"]], on = ["Date", "Symbol"], how = "left")
            rebalancePortfolio.rename(columns = {'Close' : "Price"}, inplace = True)
            rebalancePortfolio.drop(columns = ["Date", "FY"], inplace = True)
            
            currentHoldings, chargesPaid = performInitialRebalance(rebalancePortfolio.copy(), currentHoldings, capital, initialRebalancePeriod, rebalanceCounter)
            rebalanceCounter += 1
            initialRebalanceRunning = True
            nav.append({"Date" : liveDate, "NAV" : currentHoldings.Value.sum()})
            chargesPaid["Date"] = liveDate
            transactionalCharges.append(chargesPaid)

        elif initialRebalanceRunning:

            rebalancePortfolio["Date"] = liveDate
            rebalancePortfolio.drop(columns = ["Price"], inplace = True)
            rebalancePortfolio = pd.merge(rebalancePortfolio, price_data[["Date", "Symbol", "Close"]], on = ["Date", "Symbol"], how = "left")
            rebalancePortfolio.rename(columns = {'Close' : "Price"}, inplace = True)
            rebalancePortfolio.drop(columns = ["Date"], inplace = True)

            currentHoldings, chargesPaid = performInitialRebalance(rebalancePortfolio.copy(), currentHoldings, capital, initialRebalancePeriod, rebalanceCounter)
            rebalanceCounter += 1
            nav.append({"Date" : liveDate, "NAV" : currentHoldings.Value.sum()})

            chargesPaid["Date"] = liveDate
            transactionalCharges.append(chargesPaid)

            if rebalanceCounter == initialRebalancePeriod:
                initialRebalanceRunning = False
                initialRebalanceDone = True
                rebalanceCounter = 0

                currentHoldings.to_excel(writer, liveDate.strftime("%y-%m-%d"))

        elif (liveDate in rebalanceDates) and (not generalRebalanceRunning):

            rebalancePortfolio = allPortfolio[allPortfolio["Date"] == liveDate].reset_index(drop = True)
            rebalancePortfolio = pd.merge(rebalancePortfolio, price_data[["Date", "Symbol", "Close"]], on = ["Date", "Symbol"], how = "left")
            rebalancePortfolio.rename(columns = {'Close' : "Price"}, inplace = True)
            rebalancePortfolio.drop(columns = ["Date", "FY"], inplace = True)  

            currentHoldings["Date"] = liveDate
            currentHoldings = pd.merge(currentHoldings.reset_index(), price_data[["Date", "Symbol", "Close"]], on = ["Date", "Symbol"], how = "left").set_index("Symbol")
            currentHoldings.rename(columns = {'Close' : "Price"}, inplace = True)
            currentHoldings.drop(columns = ["Date"], inplace = True)

            currentHoldings, chargesPaid = generalRebalanceDay1(rebalancePortfolio.copy(), currentHoldings.copy(), rebalanceCounter)
        
            rebalanceCounter += 1

            generalRebalanceRunning = True

            nav.append({"Date" : liveDate, "NAV" : currentHoldings.Value.sum()})

            chargesPaid["Date"] = liveDate
            transactionalCharges.append(chargesPaid)


        elif generalRebalanceRunning:

            if (rebalanceCounter+1)%2 == 0:
                currentHoldings = updateDailyNAV(currentHoldings.copy(), price_data.copy(), liveDate)
                nav.append({"Date" : liveDate, "NAV" : currentHoldings.Value.sum()})
                rebalanceCounter += 1
                continue

            # rebalancePortfolio["Date"] = liveDate
            # rebalancePortfolio.drop(columns = ["Price"], inplace = True)
            # rebalancePortfolio = pd.merge(rebalancePortfolio, price_data[["Date", "Symbol", "Close", "Open"]], on = ["Date", "Symbol"], how = "left")
            # rebalancePortfolio.rename(columns = {'Close' : "Price"}, inplace = True)
            # rebalancePortfolio.drop(columns = ["Date"], inplace = True)

            currentHoldings["Date"] = liveDate
            currentHoldings = pd.merge(currentHoldings.reset_index(), price_data[["Date", "Symbol", "Close", "Open"]], on = ["Date", "Symbol"], how = "left").set_index("Symbol")
            currentHoldings.rename(columns = {'Close' : "Price"}, inplace = True)
            currentHoldings.drop(columns = ["Date"], inplace = True)

            currentHoldings = pd.merge(currentHoldings, rebalancePortfolio.set_index("Symbol")[["Weight"]], how = "left", on = "Symbol")
            currentHoldings[[ "Weight"]] = currentHoldings[[ "Weight"]].fillna(0)
            currentHoldings.loc["Cash",["Price", "Open"]] = currentHoldings.loc["Cash", "Value"]
            
            if (rebalanceCounter+1) == 1:
                currentHoldings, chargesPaid = generalRebalanceDay1(currentHoldings.copy(), rebalanceCounter)
                rebalanceCounter += 1
            elif (rebalanceCounter+1) == 3:
                currentHoldings, chargesPaid = generalRebalanceDay3Buying(currentHoldings.copy(), rebalanceCounter)
                currentHoldings, chargesPaid = generalRebalanceDay3Selling(currentHoldings.copy(), rebalanceCounter)
                rebalanceCounter += 1
            elif (rebalanceCounter+1) == 5:
                currentHoldings, chargesPaid = generalRebalanceDay3Buying(currentHoldings.copy(), rebalanceCounter)
                currentHoldings, chargesPaid = generalRebalanceDay5Selling(currentHoldings.copy(), rebalanceCounter)
                rebalanceCounter += 1
            elif (rebalanceCounter + 1) == 7:
                currentHoldings, chargesPaid = generalRebalanceDay7Buying(currentHoldings.copy(), rebalanceCounter)
                rebalanceCounter += 1
            
            nav.append({"Date" : liveDate, "NAV" : currentHoldings.Value.sum()})

            if rebalanceCounter == 7:
                generalRebalanceDone = False
                generalRebalanceRunning = False
                rebalanceCounter = 0
                currentHoldings.to_excel(writer, liveDate.strftime("%y-%m-%d"))
        else:
            currentHoldings = updateDailyNAV(currentHoldings.copy(), price_data.copy(), liveDate)
            nav.append({"Date" : liveDate, "NAV" : currentHoldings.Value.sum()})

    print(liveDate)
writer.close()

  5%|▌         | 1/20 [00:04<01:30,  4.75s/it]

2007-03-30 00:00:00


 10%|█         | 2/20 [00:11<01:42,  5.67s/it]

2008-03-31 00:00:00


 15%|█▌        | 3/20 [00:17<01:39,  5.84s/it]

...

2024-03-28 00:00:00


 95%|█████████▌| 19/20 [01:56<00:06,  6.25s/it]

2025-03-28 00:00:00


100%|██████████| 20/20 [01:58<00:00,  5.91s/it]

2025-06-06 00:00:00


In [16]:
# # Initialize variables
# rebalanceCounter = 0
# initialRebalanceRunning = False
# initialRebalanceDone = False
# generalRebalanceRunning = False
# generalRebalanceDone = False

# nav = list()
# transactionalCharges = list()

# writer = pd.ExcelWriter("Rebalances.xlsx")

# def check_and_clean_portfolio(portfolio_df, date_str=""):
#     """
#     Check for duplicate symbols and clean the portfolio DataFrame
#     """
#     if portfolio_df['Symbol'].duplicated().any():
#         print(f"Warning: Found duplicate symbols on {date_str}")
        
#         # Show which symbols are duplicated
#         duplicates = portfolio_df[portfolio_df['Symbol'].duplicated(keep=False)]
#         duplicate_symbols = duplicates['Symbol'].unique()
        
#         for symbol in duplicate_symbols:
#             print(f"Symbol '{symbol}' appears multiple times:")
#             symbol_entries = portfolio_df[portfolio_df['Symbol'] == symbol]
#             print(symbol_entries)
#             print()
        
#         # Remove duplicates - keeping the first occurrence
#         # You might want to aggregate instead depending on your needs
#         portfolio_df = portfolio_df.drop_duplicates(subset='Symbol', keep='first')
#         print(f"Removed duplicates. Portfolio now has {len(portfolio_df)} unique symbols.")
    
#     return portfolio_df

# def safe_merge_portfolio_data(portfolio_df, price_data_df, date_columns, date_value):
#     """
#     Safely merge portfolio data with price data, handling potential duplicates
#     """
#     # Filter price data for the specific date
#     price_subset = price_data_df[price_data_df["Date"] == date_value][date_columns].copy()
    
#     # Check for duplicates in price data
#     if price_subset['Symbol'].duplicated().any():
#         print(f"Warning: Duplicate symbols found in price data for {date_value}")
#         price_subset = price_subset.drop_duplicates(subset='Symbol', keep='first')
    
#     # Perform the merge
#     merged_df = pd.merge(portfolio_df, price_subset, on=["Date", "Symbol"], how="left")
    
#     return merged_df

# def update_rebalance_portfolio_prices(rebalance_portfolio, price_data_df, live_date):
#     """
#     Update portfolio with current prices safely
#     """
#     rebalance_portfolio["Date"] = live_date
#     rebalance_portfolio.drop(columns=["Price"], inplace=True, errors='ignore')
    
#     # Get price data for this date
#     price_subset = price_data_df[price_data_df["Date"] == live_date][["Date", "Symbol", "Close"]].copy()
    
#     # Remove duplicates in price data
#     if price_subset['Symbol'].duplicated().any():
#         print(f"Warning: Duplicate symbols in price data for {live_date}")
#         price_subset = price_subset.drop_duplicates(subset='Symbol', keep='first')
    
#     rebalance_portfolio = pd.merge(rebalance_portfolio, price_subset, on=["Date", "Symbol"], how="left")
#     rebalance_portfolio.rename(columns={'Close': "Price"}, inplace=True)
#     rebalance_portfolio.drop(columns=["Date"], inplace=True)
    
#     return rebalance_portfolio

# # Main rebalancing loop
# for fyYear in tqdm(financialYears):
#     fyDates = price_data[price_data["FY"] == fyYear]["Date"].unique().tolist()

#     for liveDate in fyDates:
#         try:
#             # Phase 1: Initial Rebalance Start
#             if (liveDate in rebalanceDates) and (not initialRebalanceDone) and (rebalanceCounter == 0):
#                 print(f"Starting initial rebalance on {liveDate}")
                
#                 rebalancePortfolio = allPortfolio[allPortfolio["Date"] == liveDate].reset_index(drop=True)
#                 rebalancePortfolio = safe_merge_portfolio_data(
#                     rebalancePortfolio, 
#                     price_data, 
#                     ["Date", "Symbol", "Close"], 
#                     liveDate
#                 )
#                 rebalancePortfolio.rename(columns={'Close': "Price"}, inplace=True)
#                 rebalancePortfolio.drop(columns=["Date", "FY"], inplace=True)
                
#                 # Clean portfolio of duplicates
#                 rebalancePortfolio = check_and_clean_portfolio(rebalancePortfolio, str(liveDate))
                
#                 currentHoldings, chargesPaid = performInitialRebalance(
#                     rebalancePortfolio.copy(), 
#                     currentHoldings, 
#                     capital, 
#                     initialRebalancePeriod, 
#                     rebalanceCounter
#                 )
#                 rebalanceCounter += 1
#                 initialRebalanceRunning = True
#                 nav.append({"Date": liveDate, "NAV": currentHoldings.Value.sum()})
#                 chargesPaid["Date"] = liveDate
#                 transactionalCharges.append(chargesPaid)

#             # Phase 2: Continue Initial Rebalance
#             elif initialRebalanceRunning:
#                 print(f"Continuing initial rebalance on {liveDate}, counter: {rebalanceCounter}")
                
#                 rebalancePortfolio = update_rebalance_portfolio_prices(rebalancePortfolio, price_data, liveDate)
                
#                 # Clean portfolio of duplicates
#                 rebalancePortfolio = check_and_clean_portfolio(rebalancePortfolio, str(liveDate))

#                 currentHoldings, chargesPaid = performInitialRebalance(
#                     rebalancePortfolio.copy(), 
#                     currentHoldings, 
#                     capital, 
#                     initialRebalancePeriod, 
#                     rebalanceCounter
#                 )
#                 rebalanceCounter += 1
#                 nav.append({"Date": liveDate, "NAV": currentHoldings.Value.sum()})

#                 chargesPaid["Date"] = liveDate
#                 transactionalCharges.append(chargesPaid)

#                 if rebalanceCounter == initialRebalancePeriod:
#                     initialRebalanceRunning = False
#                     initialRebalanceDone = True
#                     rebalanceCounter = 0
#                     print(f"Initial rebalance completed on {liveDate}")

#                     currentHoldings.to_excel(writer, liveDate.strftime("%y-%m-%d"))

#             # Phase 3: Start General Rebalance
#             elif (liveDate in rebalanceDates) and (not generalRebalanceRunning):
#                 print(f"Starting general rebalance on {liveDate}")
                
#                 rebalancePortfolio = allPortfolio[allPortfolio["Date"] == liveDate].reset_index(drop=True)
#                 rebalancePortfolio = safe_merge_portfolio_data(
#                     rebalancePortfolio, 
#                     price_data, 
#                     ["Date", "Symbol", "Close"], 
#                     liveDate
#                 )
#                 rebalancePortfolio.rename(columns={'Close': "Price"}, inplace=True)
#                 rebalancePortfolio.drop(columns=["Date", "FY"], inplace=True)
                
#                 # Clean portfolio of duplicates
#                 rebalancePortfolio = check_and_clean_portfolio(rebalancePortfolio, str(liveDate))

#                 currentHoldings["Date"] = liveDate
#                 currentHoldings_reset = currentHoldings.reset_index()
                
#                 # Get price data and remove duplicates
#                 price_subset = price_data[price_data["Date"] == liveDate][["Date", "Symbol", "Close"]].copy()
#                 if price_subset['Symbol'].duplicated().any():
#                     print(f"Warning: Duplicate symbols in price data for current holdings on {liveDate}")
#                     price_subset = price_subset.drop_duplicates(subset='Symbol', keep='first')
                
#                 currentHoldings = pd.merge(currentHoldings_reset, price_subset, on=["Date", "Symbol"], how="left").set_index("Symbol")
#                 currentHoldings.rename(columns={'Close': "Price"}, inplace=True)
#                 currentHoldings.drop(columns=["Date"], inplace=True)

#                 currentHoldings, chargesPaid = generalRebalanceDay1(rebalancePortfolio.copy(), currentHoldings.copy(), rebalanceCounter)
            
#                 rebalanceCounter += 1
#                 generalRebalanceRunning = True

#                 nav.append({"Date": liveDate, "NAV": currentHoldings.Value.sum()})
#                 chargesPaid["Date"] = liveDate
#                 transactionalCharges.append(chargesPaid)

#             # Phase 4: Continue General Rebalance
#             elif generalRebalanceRunning:
#                 print(f"Continuing general rebalance on {liveDate}, counter: {rebalanceCounter}")
                
#                 if (rebalanceCounter + 1) % 2 == 0:
#                     currentHoldings = updateDailyNAV(currentHoldings.copy(), price_data.copy(), liveDate)
#                     nav.append({"Date": liveDate, "NAV": currentHoldings.Value.sum()})
#                     rebalanceCounter += 1
#                     continue

#                 currentHoldings["Date"] = liveDate
#                 currentHoldings_reset = currentHoldings.reset_index()
                
#                 # Get price data with both Close and Open prices
#                 price_subset = price_data[price_data["Date"] == liveDate][["Date", "Symbol", "Close", "Open"]].copy()
#                 if price_subset['Symbol'].duplicated().any():
#                     print(f"Warning: Duplicate symbols in price data for general rebalance on {liveDate}")
#                     price_subset = price_subset.drop_duplicates(subset='Symbol', keep='first')
                
#                 currentHoldings = pd.merge(currentHoldings_reset, price_subset, on=["Date", "Symbol"], how="left").set_index("Symbol")
#                 currentHoldings.rename(columns={'Close': "Price"}, inplace=True)
#                 currentHoldings.drop(columns=["Date"], inplace=True)

#                 # Merge with rebalance portfolio weights
#                 rebalance_weights = rebalancePortfolio.set_index("Symbol")[["Weight"]]
#                 currentHoldings = pd.merge(currentHoldings, rebalance_weights, how="left", on="Symbol")
#                 currentHoldings[["Weight"]] = currentHoldings[["Weight"]].fillna(0)
                
#                 # Handle Cash row
#                 if "Cash" in currentHoldings.index:
#                     currentHoldings.loc["Cash", ["Price", "Open"]] = currentHoldings.loc["Cash", "Value"]
                
#                 # Execute appropriate rebalance function based on counter
#                 if (rebalanceCounter + 1) == 1:
#                     currentHoldings, chargesPaid = generalRebalanceDay1(currentHoldings.copy(), rebalanceCounter)
#                     rebalanceCounter += 1
#                 elif (rebalanceCounter + 1) == 3:
#                     currentHoldings, chargesPaid = generalRebalanceDay3Buying(currentHoldings.copy(), rebalanceCounter)
#                     currentHoldings, chargesPaid = generalRebalanceDay3Selling(currentHoldings.copy(), rebalanceCounter)
#                     rebalanceCounter += 1
#                 elif (rebalanceCounter + 1) == 5:
#                     currentHoldings, chargesPaid = generalRebalanceDay3Buying(currentHoldings.copy(), rebalanceCounter)
#                     currentHoldings, chargesPaid = generalRebalanceDay5Selling(currentHoldings.copy(), rebalanceCounter)
#                     rebalanceCounter += 1
#                 elif (rebalanceCounter + 1) == 7:
#                     currentHoldings, chargesPaid = generalRebalanceDay7Buying(currentHoldings.copy(), rebalanceCounter)
#                     rebalanceCounter += 1
                
#                 nav.append({"Date": liveDate, "NAV": currentHoldings.Value.sum()})

#                 if rebalanceCounter == 7:
#                     generalRebalanceDone = False
#                     generalRebalanceRunning = False
#                     rebalanceCounter = 0
#                     print(f"General rebalance completed on {liveDate}")
#                     currentHoldings.to_excel(writer, liveDate.strftime("%y-%m-%d"))
                    
#             # Phase 5: Regular NAV Update
#             else:
#                 currentHoldings = updateDailyNAV(currentHoldings.copy(), price_data.copy(), liveDate)
#                 nav.append({"Date": liveDate, "NAV": currentHoldings.Value.sum()})

#         except Exception as e:
#             print(f"Error processing date {liveDate}: {str(e)}")
#             print(f"Error type: {type(e).__name__}")
            
#             # Add additional debugging information
#             if 'rebalancePortfolio' in locals():
#                 print(f"RebalancePortfolio shape: {rebalancePortfolio.shape}")
#                 if 'Symbol' in rebalancePortfolio.columns:
#                     print(f"Unique symbols: {rebalancePortfolio['Symbol'].nunique()}")
#                     print(f"Total rows: {len(rebalancePortfolio)}")
                    
#             if 'currentHoldings' in locals():
#                 print(f"CurrentHoldings index: {currentHoldings.index.tolist()}")
#                 print(f"CurrentHoldings shape: {currentHoldings.shape}")
            
#             # Re-raise the exception to stop execution
#             raise e

#     print(f"Completed processing for FY {fyYear}, last date: {liveDate}")

# writer.close()
# print("Rebalancing process completed successfully!")

# # Create final outputs
# nav_df = pd.DataFrame(nav)
# charges_df = pd.DataFrame(transactionalCharges)

# print(f"Final NAV DataFrame shape: {nav_df.shape}")
# print(f"Final Charges DataFrame shape: {charges_df.shape}")

# # Save additional outputs
# nav_df.to_excel("NAV_History.xlsx", index=False)
# charges_df.to_excel("Transaction_Charges.xlsx", index=False)

# print("All outputs saved successfully!")

In [46]:
finalnav = pd.DataFrame(nav).set_index("Date")
finalnav = finalnav.pct_change().add(1).cumprod().fillna(1)*100
finalnav
finalnav.rename(columns = {"NAV" : "Value"}, inplace = True)

In [47]:
# finalnav = finalnav.drop(index='2022-03-07')
(finalnav.div(finalnav.cummax())-1).min()

Value   -0.625577
dtype: float64

In [48]:
def dd(ser):
    return round((ser["Value"] / ser["Value"].cummax() - 1).min()*100, 2)

def std(ser):
    return round(ser["Value"].pct_change().std()*np.sqrt(252)*100, 2)

def cagr(ser):
    dur = (ser["Date"].iloc[-1] - ser["Date"].iloc[0]).days/365
    return round(((ser["Value"].iloc[-1]/ser["Value"].iloc[0])**(1/dur)-1)*100, 2)

In [49]:
print("\033[Result for MIDCAP_150_6MMcap_VolW_TOP_25_FREQ_2 ALPHA model in new deployment model.\033[0m")
print("\033[1mNAV: \033[0m", round(finalnav.Value.iloc[-1],2))
print("\033[1mCAGR: \033[0m", cagr(finalnav.reset_index()))
print("\033[1mDD: \033[0m", dd(finalnav))
print("\033[1mSTD: \033[0m", std(finalnav))

esult for MIDCAP_150_6MMcap_VolW_TOP_25_FREQ_2 ALPHA model in new deployment model.
NAV:  1618.33
CAGR:  15.8
DD:  -62.56
STD:  18.43


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

# Fetch NSE 500 data from yfinance
nse500 = yf.download("^CRSLDX", start=finalnav.index.min(), end=finalnav.index.max())['Close']

# Ensure alignment of finalnav and NSE 500
finalnav = finalnav.copy()  # Prevent accidental modification elsewhere
nse500 = nse500.reindex(finalnav.index).fillna(method='ffill')  # Align NSE 500 with finalnav

# Normalize NSE 500 for fair comparison
nse500_normalized = (nse500 / nse500.iloc[0] * finalnav['Value'].iloc[0]).to_frame(name='Value')

# Bold text for the plot title and annotations
bold_start = ""
bold_end = ""

# Create the main plot
plt.figure(figsize=(12, 6))
plt.plot(finalnav.index, finalnav['Value'], label="Model NAV", color='blue', linewidth=2)
plt.plot(nse500_normalized.index, nse500_normalized['Value'], label="NSE 500", color='orange', linestyle='--', linewidth=2)

# Add title and labels
plt.title(f"{bold_start}Jensens Alpha Model NAV vs NSE 500 Performance{bold_end}", fontsize=14, weight='bold')
plt.xlabel("Date", fontsize=12)
plt.ylabel("NAV", fontsize=12)

# Calculate metrics for the entire period
nav = round(finalnav['Value'].iloc[-1], 2)
cagr_value = cagr(finalnav.reset_index())  # Assuming `cagr` is defined
dd_value = dd(finalnav)                    # Assuming `dd` is defined
std_value = std(finalnav)                  # Assuming `std` is defined

# Calculate metrics for NSE 500
nse_nav = round(nse500_normalized['Value'].iloc[-1], 2)
nse_cagr_value = cagr(nse500_normalized.reset_index())
nse_dd_value = dd(nse500_normalized)
nse_std_value = std(nse500_normalized)

# Annotate metrics on the top-left corner of the plot
textstr = (
    f"{bold_start}Model Metrics:{bold_end}\n"
    f"NAV: {nav}\n"
    f"CAGR: {cagr_value:.2f}%\n"
    f"DD: {dd_value:.2f}%\n"
    f"STD: {std_value:.2f}%\n\n"
    f"{bold_start}NSE 500 Metrics:{bold_end}\n"
    f"NAV: {nse_nav}\n"
    f"CAGR: {nse_cagr_value:.2f}%\n"
    f"DD: {nse_dd_value:.2f}%\n"
    f"STD: {nse_std_value:.2f}%"
)
plt.gca().text(0.01, 0.95, textstr, transform=plt.gca().transAxes, fontsize=10,
               verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", edgecolor="black"))

# Add legend and grid
plt.legend(loc="upper left", fontsize=10)
plt.grid(alpha=0.3)

# Tight layout and show plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

# Fetch NSE 500 data from yfinance
nse500 = yf.download("^CRSLDX", start=finalnav.index.min(), end=finalnav.index.max())['Close']

# Ensure alignment of finalnav and NSE 500
finalnav = finalnav.copy()  # Ensure it's not modified elsewhere
nse500 = nse500.reindex(finalnav.index).fillna(method='ffill')  # Align NSE 500 with finalnav

# Normalize NSE 500 for fair comparison
nse500_normalized = (nse500 / nse500.iloc[0] * finalnav['Value'].iloc[0]).to_frame(name='Value')

# Define the number of segments
n_parts = 5
data_splits = np.array_split(finalnav, n_parts)
nse500_splits = np.array_split(nse500_normalized, n_parts)

# Bold text for the plot title and annotations
bold_start = ""
bold_end = ""

# Create subplots for each segment
fig, axes = plt.subplots(n_parts, 1, figsize=(12, 18))

# Plot each segment
for i, (ax, final_part, nse_part) in enumerate(zip(axes, data_splits, nse500_splits)):
    # Calculate metrics for the segment
    nav = round(final_part['Value'].iloc[-1], 2)
    cagr_value = cagr(final_part.reset_index())  # Assuming `cagr` is defined
    dd_value = dd(final_part)                    # Assuming `dd` is defined
    std_value = std(final_part)                  # Assuming `std` is defined

    # Calculate metrics for NSE 500 segment
    nse_nav = round(nse_part['Value'].iloc[-1], 2)
    nse_cagr_value = cagr(nse_part.reset_index())
    nse_dd_value = dd(nse_part)
    nse_std_value = std(nse_part)

    # Plot finalnav and NSE 500
    ax.plot(final_part.index, final_part['Value'], label=f"Model NAV (Segment {i + 1})", color='blue', linewidth=2)
    ax.plot(nse_part.index, nse_part['Value'], label="NSE 500", color='orange', linewidth=2, linestyle='--')

    # Set titles and labels
    ax.set_title(f"Segment {i + 1} NAV Performance", fontsize=12, weight='bold')
    ax.set_ylabel("NAV", fontsize=10)

    # Annotate metrics for the segment
    textstr = (
        f"{bold_start}Model Metrics (Segment {i + 1}):{bold_end}\n"
        f"NAV: {nav}\n"
        f"CAGR: {cagr_value:.2f}%\n"
        f"DD: {dd_value:.2f}%\n"
        f"STD: {std_value:.2f}%\n\n"
        f"{bold_start}NSE 500 Metrics:{bold_end}\n"
        f"NAV: {nse_nav}\n"
        f"CAGR: {nse_cagr_value:.2f}%\n"
        f"DD: {nse_dd_value:.2f}%\n"
        f"STD: {nse_std_value:.2f}%"
    )
    ax.text(0.01, 0.95, textstr, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", edgecolor="black"))

    # Add legend and grid
    ax.legend(loc="upper left", fontsize=9)
    ax.grid(alpha=0.3)

# Set x-axis label for the bottom subplot
axes[-1].set_xlabel("Date", fontsize=12)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# finalnav.to_csv('NAVMULTI.csv')

In [50]:
allPortfolio

,Date,Symbol,Weight,FY
0,2006-06-19,63MOONS,0.02,FY2006-2007
1,2006-06-19,ITC,0.02,FY2006-2007
2,2006-06-19,LUPIN,0.02,FY2006-2007
3,2006-06-19,M&M,0.02,FY2006-2007
4,2006-06-19,NATIONALUM,0.02,FY2006-2007
...,...,...,...,...
811,2025-04-08,GUJGASLTD,0.02,FY2025-2026
812,2025-04-08,HDFCBANK,0.02,FY2025-2026
813,2025-04-08,HEROMOTOCO,0.02,FY2025-2026
814,2025-04-08,COLPAL,0.02,FY2025-2026


In [32]:
finalnav

,Value
Date,
2006-06-19,100.000000
2006-06-20,99.246918
2006-06-21,100.294108
2006-06-22,101.940325
2006-06-23,102.679506
...,...
2025-06-02,1406.545211
2025-06-03,1403.471772
2025-06-04,1410.376654


In [51]:
# Filter from 1st Jan 2014
nav_filtered = finalnav[finalnav.index >= '2010-10-01'].copy()

# Rebase NAV to start from 100
base = nav_filtered['Value'].iloc[0]
nav_filtered['Rebased_NAV'] = (nav_filtered['Value'] / base) * 100
nav_filtered

,Value,Rebased_NAV
Date,,
2010-10-01,244.509851,100.000000
2010-10-04,246.049644,100.629747
2010-10-05,246.748658,100.915630
2010-10-06,248.858385,101.778470
2010-10-07,247.601857,101.264573
...,...,...
2025-06-02,1595.146545,652.385389
2025-06-03,1591.668637,650.962989
2025-06-04,1599.503457,654.167285


In [52]:
def dd(ser):
    return round((ser["Rebased_NAV"] / ser["Rebased_NAV"].cummax() - 1).min()*100, 2)

def std(ser):
    return round(ser["Rebased_NAV"].pct_change().std()*np.sqrt(252)*100, 2)

def cagr(ser):
    dur = (ser["Date"].iloc[-1] - ser["Date"].iloc[0]).days/365
    return round(((ser["Rebased_NAV"].iloc[-1]/ser["Rebased_NAV"].iloc[0])**(1/dur)-1)*100, 2)

print("\033[Result for MCAP_500_TOP_25_FREQ_2 ALPHA model in new deployment model.\033[0m")
print("\033[1mNAV: \033[0m", round(nav_filtered.Rebased_NAV.iloc[-1],2))
print("\033[1mCAGR: \033[0m", cagr(nav_filtered.reset_index()))
print("\033[1mDD: \033[0m", dd(nav_filtered))
print("\033[1mSTD: \033[0m", std(nav_filtered))

esult for MCAP_500_TOP_25_FREQ_2 ALPHA model in new deployment model.
NAV:  661.87
CAGR:  13.73
DD:  -39.96
STD:  14.67


In [35]:
# Get last NAV of each year
year_end = nav_filtered['Rebased_NAV'].resample('Y').last()
year_start = nav_filtered['Rebased_NAV'].resample('Y').first()

cy_returns = (year_end / year_start - 1) * 100
cy_returns.index = cy_returns.index.year  # Convert to year only
print("📆 Calendar Year Returns:\n", cy_returns.round(2))

📆 Calendar Year Returns:
 Date
2010    -3.44
2011   -17.11
2012    29.98
2013     4.22
2014    49.95
2015     0.25
2016    13.29
2017    36.02
2018   -11.06
2019    -0.04
2020    11.52
2021    39.12
2022    -2.48
2023    40.62
2024    26.12
2025    -5.04
Name: Rebased_NAV, dtype: float64


In [53]:
# Shift year to start in April
fy_end = nav_filtered['Rebased_NAV'].resample('M').last().shift(-3).resample('A-MAR').last()
fy_start = nav_filtered['Rebased_NAV'].resample('M').last().shift(-3).resample('A-MAR').first()

fy_returns = (fy_end / fy_start - 1) * 100
fy_returns.index = fy_returns.index.year  # FY ending year
print("📊 Fiscal Year Returns:\n", fy_returns.round(2))

📊 Fiscal Year Returns:
 Date
2011     4.92
2012    -0.30
2013     9.28
2014    49.16
2015    15.50
2016     4.81
2017    17.74
2018     3.95
2019    -7.34
2020    -1.19
2021    50.39
2022    -3.74
2023    12.94
2024    48.59
2025    -6.69
2026      NaN
Name: Rebased_NAV, dtype: float64


In [54]:

nav = nav_filtered['Rebased_NAV'].copy()
running_max = nav.cummax()
drawdown = (nav - running_max) / running_max

# Identify drawdown periods
in_drawdown = drawdown < 0
drawdown_periods = (in_drawdown != in_drawdown.shift()).cumsum() * in_drawdown

# Collect MDD for each drawdown period
mdd_by_period = drawdown.groupby(drawdown_periods).min()

# Drop zero index (non-drawdown periods)
mdd_by_period = mdd_by_period[mdd_by_period != 0]

# Get worst and second-worst drawdowns
sorted_mdd = mdd_by_period.sort_values()

worst = sorted_mdd.iloc[0]
second_worst = sorted_mdd.iloc[1] if len(sorted_mdd) > 1 else None

print(f"📉 Worst Drawdown: {worst * 100:.2f}%")
if second_worst is not None:
    print(f"📉 Second-Worst Drawdown: {second_worst * 100:.2f}%")
else:
    print("Only one drawdown period detected.")


📉 Worst Drawdown: -39.96%
📉 Second-Worst Drawdown: -22.50%


In [353]:
allPortfolio

,Date,Symbol,Weight,FY
0,2006-06-19,CHEMPLASTS,NaN,FY2006-2007
1,2006-06-19,BAJAJHIND,NaN,FY2006-2007
2,2006-06-19,IVRCLINFRA,NaN,FY2006-2007
3,2006-06-19,HEROMOTOCO,NaN,FY2006-2007
4,2006-06-19,ESSAROIL,NaN,FY2006-2007
...,...,...,...,...
1870,2024-12-18,NEULANDLAB,0.057366,FY2024-2025
1871,2024-12-18,PGEL,0.060894,FY2024-2025
1872,2024-12-18,VOLTAS,0.030672,FY2024-2025
1873,2024-12-18,BSE,0.039104,FY2024-2025


In [ ]:
## NOtes
# 1) Next day if am rebalancing, how to determine the weights of current port, 
#  should we consider previous days weight and buy or assume current  days price and calculate the buy qty (In initial rebalance phase, after 1st day)




In [ ]:
import pandas as pd
from datetime import datetime

def calculate_capital_gains(holdings_df, sell_info):
    """
    Calculate STCG and LTCG for the sold securities based on holding duration and selling price.
    
    :param holdings_df: DataFrame containing the current holdings with columns ['Date', 'Symbol', 'Quantity', 'Price']
    :param sell_info: DataFrame with columns ['Symbol', 'Quantity', 'Sell_Price'] containing information on the quantity of symbols to be sold and their selling prices.
    
    :return: DataFrame with columns ['Symbol', 'STCG', 'LTCG'] showing the capital gains for each symbol.
    """
    
    # Convert the Date column to datetime
    holdings_df['Date'] = pd.to_datetime(holdings_df['Date'])
    
    # Initialize lists to store results
    results = {'Symbol': [], 'STCG': [], 'LTCG': []}
    
    # Iterate through each row in the sell_info
    for _, sell_row in sell_info.iterrows():
        symbol = sell_row['Symbol']
        sell_quantity = sell_row['Quantity']
        sell_price = sell_row['Sell_Price']
        
        symbol_holdings = holdings_df[holdings_df['Symbol'] == symbol].sort_values(by='Date')
        stcg, ltcg = 0, 0
        
        # Sell securities on a FIFO basis
        for i, row in symbol_holdings.iterrows():
            if sell_quantity <= 0:
                break
            
            # Determine the quantity to sell from this lot
            qty_to_sell = min(row['Quantity'], sell_quantity)
            
            # Calculate the holding period
            holding_period = (datetime.now() - row['Date']).days
            
            # Calculate gain per share
            gain_per_share = sell_price - row['Price']
            
            if holding_period > 365:
                # Long-Term Capital Gains
                ltcg += qty_to_sell * gain_per_share * 0.125
            else:
                # Short-Term Capital Gains
                stcg += qty_to_sell * gain_per_share * 0.20
                
            # Update the remaining quantity to be sold
            sell_quantity -= qty_to_sell
            
        results['Symbol'].append(symbol)
        results['STCG'].append(stcg)
        results['LTCG'].append(ltcg)
    
    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)

    
    return results_df


In [ ]:
print(type(liveDate))
print(type(price_data["Date"].iloc[0]))
print(liveDate)
print(price_data["Date"].iloc[0])

In [ ]:
holdings_df = pd.DataFrame({
    'Date': ['2022-01-01', '2024-01-01'],
    'Symbol': ['AAPL', 'AAPL'],
    'Quantity': [10, 5],
    'Price': [150, 200]
})

sell_info = pd.DataFrame({
    'Symbol': ['AAPL'],
    'Quantity': [12],
    'Sell_Price': [250]
})

gains_df = calculate_capital_gains(holdings_df, sell_info)
print(gains_df)
